In [1]:
# !pip install tensorflow-gpu==1.14 keras==2.1.0
# !pip install tqdm
# !pip install treepoem RandomWords wordcloud tqdm Shapely opencv-python ghostscript

In [2]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from PIL import Image
import cv2
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
from barcode import BarcodeConfig

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ec2-user/a

In [3]:
class InferenceConfig(BarcodeConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [4]:
model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [5]:
weights_path = model.find_last()
model.load_weights(weights_path, by_name=True)

Re-starting from epoch 100


In [6]:
test_ind = 90000
test_image_path = "/home/ec2-user/SageMaker/benchmarks/dataset/X/train_{}.png".format(test_ind)
test_image = skimage.io.imread(test_image_path)
test_image = test_image.reshape(test_image.shape[0], test_image.shape[1], 1)

In [7]:
r = model.detect([test_image], verbose=1)[0]

Processing 1 images
image                    shape: (8451, 8451, 1)       min:    0.00000  max:  237.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  133.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 8451.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


In [8]:
mask = r['masks']

In [13]:
mask.squeeze().()


False

In [5]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 15))
plt.imshow(test_image.squeeze(), cmap='gray')

NameError: name 'test_image' is not defined

<Figure size 1080x1080 with 0 Axes>

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(mask.squeeze(), cmap='gray')

In [13]:
#obtain confidence scores
def get_confidence_score(im_mask, bb):
    cnt = 0
    pixel_acc = 0
    im_w = im_mask.shape[0]
    im_h = im_mask.shape[1]
    poly = Polygon(bb)
    minx, miny, maxx, maxy = list(np.int_(poly.bounds))
    for row in range(minx, min(minx + 400, maxx)):
        for col in range(miny, min(miny + 400, maxy)):
            p_temp = Point(row, col)
            if p_temp.within(poly):
                cnt += 1
                pixel_acc = pixel_acc + im_mask[col, row]
    if cnt == 0:
        return 0
    avg_pix_value = pixel_acc/cnt
    return avg_pix_value

# Finds all bounding boxes using contours in a mask
def findBoundingboxs(image):
    contours, _ = cv2.findContours(image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boundingBoxs = []
    scores = []
    for contour in contours:
        rect = cv2.minAreaRect(contour)
        poly = cv2.boxPoints(rect)
        box = np.int0(poly)
        boundingBoxs.append(box)
        scores.append(get_confidence_score(image, box))
        
    return boundingBoxs, scores

# Calculate Intersection over Union of two bounding boxes
def iou(bb1, bb2):
    a = Polygon(bb1)
    b = Polygon(bb2)
    return a.intersection(b).area / a.union(b).area

# Calculates the mean Intersection over Union of two masks
# Only considers the number of BBs in label masks
# algo may needs update
def mean_iou(pred_bbs, label_bbs):
    if len(pred_bbs) == 0:
        return 0
    iou_scores = np.zeros((len(label_bbs), len(pred_bbs)))
    for i in range(len(label_bbs)):
        for j in range(len(pred_bbs)):
            iou_scores[i, j] = iou(label_bbs[i], pred_bbs[j])

    scores_taken = np.amax(iou_scores, axis=1)
    assert(len(scores_taken) == len(label_bbs))
    return sum(scores_taken) / len(label_bbs)

In [ ]:
def test(model, threshold=0.5):
    # remove extensions
    total_iou = 0
    all_data = {"imageName": [], "confidenceScore": [], 
                "predBoundingBox": [], "truthBoundingBox": [],
                "iou": []}
    total_num_GT = 0
    for i in tqdm(range(90000, 90050)):
        # load and pad images
        test_image_path = "/home/ec2-user/SageMaker/benchmarks/dataset/X/train_{}.png".format(i)
        test_image = skimage.io.imread(test_image_path)
        test_image = test_image.reshape(test_image.shape[0], test_image.shape[1], 1)
        test_gt_path = "/home/ec2-user/SageMaker/benchmarks/dataset/Y/train_mask_{}.png".format(i)
        test_gt = skimage.io.imread(test_gt_path)
        r = model.detect([test_image], verbose=1)[0]
        prediction = r['masks'].squeeze()
        # find prediction boundingboxs
        prediction = (prediction > threshold).astype(np.uint8)
#         _, prediction = cv2.threshold(prediction, 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
        bbs_pred, confidences = findBoundingboxs(prediction)
        bbs_truth, _ = findBoundingboxs(test_gt)
        
        # constructing database
        all_data["imageName"] += [i] * len(confidences)
        all_data["confidenceScore"] += confidences
        all_data["predBoundingBox"] += bbs_pred
        for bb_pred in bbs_pred:
            max_iou = 0
            max_bb = None
            for bb_truth in bbs_truth:
                this_iou = iou(bb_pred, bb_truth)
                if this_iou > max_iou:
                    max_iou = this_iou
                    max_bb = bb_truth
            all_data["iou"].append(max_iou)
            all_data["truthBoundingBox"].append(max_bb)
        curr_iou = mean_iou(bbs_pred, bbs_truth)
        total_iou += curr_iou
        total_num_GT += len(bbs_truth)
    total_iou /= 90050 - 90000
    return total_iou, all_data, total_num_GT

In [7]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import json
import glob
import math
from tqdm.notebook import tqdm
import pandas as pd

ModuleNotFoundError: No module named 'shapely'

In [ ]:
result, all_data, total_num_GT = test(model, 0.2)
print("Mean IoU: ", result)

In [ ]:
iou_threshold = 60 / 100

data = pd.DataFrame(all_data)
data["TP"] = (data["iou"] >= iou_threshold) * 1
data["FP"] = (data["iou"] < iou_threshold) * 1
data = data.sort_values(by=["confidenceScore", "TP"], ascending=[False, False])
data["Acc TP"] = np.cumsum(data["TP"])
data["Acc FP"] = np.cumsum(data["FP"])
data["Precision"] = data["Acc TP"] / (data["Acc TP"] + data["Acc FP"])
data["Recall"] = data["Acc TP"] / total_num_GT
plt.plot(list(data["Recall"]), list(data["Precision"]))
interpolation = list(data["Precision"])
for i in reversed(range(len(interpolation) - 1)):
    if interpolation[i+1]>interpolation[i]:
        interpolation[i]=interpolation[i+1]
# plt.figsize((15, 10))
plt.plot(list(data["Recall"]), interpolation)
from sklearn import metrics
print("Average Precision:", metrics.auc(list(data["Recall"]), interpolation))

In [ ]:
data

In [ ]:
a = np.array([1,2,3])
a[a > 2]

In [ ]:
# Calculate Intersection over Union of two images
def iou(im1, im2):
    sums = im1 + im2
    intersection = len(sums[sums == 2])
    union = len(sums[sums >= 1])
    if union == 0:
        return 0
    return intersection / union

In [ ]:
iou(mask.squeeze(), gt)

In [ ]:
def test(model, num_tests, threshold=0.5):
    # remove extensions
    total_iou = 0
    all_data = {"imageName": [], "confidenceScore": [], 
                "iou": []}
    total_num_GT = 0
    for i in tqdm(range(90000, 90000+num_tests)):
        # load and pad images
        test_image_path = "/home/ec2-user/SageMaker/benchmarks/dataset/X/train_{}.png".format(i)
        test_image = skimage.io.imread(test_image_path)
        test_image = test_image.reshape(test_image.shape[0], test_image.shape[1], 1)
        test_gt_path = "/home/ec2-user/SageMaker/benchmarks/dataset/Y/train_mask_{}.png".format(i)
        test_gt = skimage.io.imread(test_gt_path)
        r = model.detect([test_image])[0]
        if r['masks'].shape[-1] > 0 and r['masks'].shape[0] == r['masks'].shape[1]:
            prediction = r['masks'][:, :, 0]
        else:
            num_tests -= 1
            continue
        # find prediction boundingboxs
        prediction = (prediction > threshold).astype(np.uint8)
        curr_iou = iou(prediction, test_gt)
        
        # constructing database
        if curr_iou == 0:
            num_tests -= 1
            continue
        all_data["imageName"] += [i] 
        if len(r['scores']) > 0:
            all_data["confidenceScore"] += [r['scores'][0]]
        else:
            all_data["confidenceScore"] += [0]
        all_data["iou"].append(curr_iou)
        total_iou += curr_iou
    total_iou /= num_tests
    return total_iou, all_data

In [ ]:
result, all_data = test(model, 1000, threshold=0.3)
print("Mean IoU: ", result)

In [ ]:
iou_threshold = 50 / 100

data = pd.DataFrame(all_data)
data["TP"] = (data["iou"] >= iou_threshold) * 1
data["FP"] = (data["iou"] < iou_threshold) * 1
data = data.sort_values(by=["confidenceScore", "TP"], ascending=[False, False])
data["Acc TP"] = np.cumsum(data["TP"])
data["Acc FP"] = np.cumsum(data["FP"])
data["Precision"] = data["Acc TP"] / (data["Acc TP"] + data["Acc FP"])
data["Recall"] = data["Acc TP"] / 10
plt.plot(list(data["Recall"]), list(data["Precision"]))
interpolation = list(data["Precision"])
for i in reversed(range(len(interpolation) - 1)):
    if interpolation[i+1]>interpolation[i]:
        interpolation[i]=interpolation[i+1]
# plt.figsize((15, 10))
plt.plot(list(data["Recall"]), interpolation)
from sklearn import metrics
print("Average Precision:", metrics.auc(list(data["Recall"]), interpolation))

In [ ]:
iou_threshold = 60 / 100

data = pd.DataFrame(all_data)
data["TP"] = (data["iou"] >= iou_threshold) * 1
data["FP"] = (data["iou"] < iou_threshold) * 1
data = data.sort_values(by=["confidenceScore", "TP"], ascending=[False, False])
data["Acc TP"] = np.cumsum(data["TP"])
data["Acc FP"] = np.cumsum(data["FP"])
data["Precision"] = data["Acc TP"] / (data["Acc TP"] + data["Acc FP"])
data["Recall"] = data["Acc TP"] / 10
plt.plot(list(data["Recall"]), list(data["Precision"]))
interpolation = list(data["Precision"])
for i in reversed(range(len(interpolation) - 1)):
    if interpolation[i+1]>interpolation[i]:
        interpolation[i]=interpolation[i+1]
# plt.figsize((15, 10))
plt.plot(list(data["Recall"]), interpolation)
from sklearn import metrics
print("Average Precision:", metrics.auc(list(data["Recall"]), interpolation))

In [ ]:
test_gt_path = "/home/ec2-user/SageMaker/benchmarks/dataset/Y/train_mask_{}.png".format(5)
test_gt = skimage.io.imread(test_gt_path)

In [ ]:
np.max(mask.squeeze().astype(np.uint8))

In [ ]:
plt.imshow(test_gt)

In [ ]:
test_ind = 90005
test_image_path = "/home/ec2-user/SageMaker/benchmarks/dataset/X/train_{}.png".format(test_ind)
test_image = skimage.io.imread(test_image_path)
test_image = test_image.reshape(test_image.shape[0], test_image.shape[1], 1)
r = model.detect([test_image], verbose=1)[0]

In [ ]:
mask= r['masks'][0]

In [ ]:
mask.shape[0]

In [ ]:
data

In [ ]:
import time
a = time.time()
for i in range(10):
    r = model.detect([test_image])[0]
b = time.time()
print(b - a)